In [1]:
import os
import torch
import torchaudio
import sys

sys.path.append('./src_torch')

import soundfile as sf

from src_torch.separation import FastMNMF2
from src_torch.Base import MultiSTFT


In [2]:
n_source = 5
n_basis = 64
device = "cuda" if torch.cuda.is_available() else "cpu"
init_SCM = "twostep"
n_bit = 64
algo = "IP"
n_iter_init = 30
g_eps = 5e-2

input_fname = "./test_audio.mp3"
n_mic = 8
n_fft = 1024
n_iter = 100


separater = FastMNMF2(
    n_source=n_source,
    n_basis=n_basis,
    device=device,
    init_SCM=init_SCM,
    n_bit=n_bit,
    algo=algo,
    n_iter_init=n_iter_init,
    g_eps=g_eps,
)

wav, sample_rate = torchaudio.load(input_fname, channels_first=False)
wav /= torch.abs(wav).max() * 1.2
M = min(len(wav), n_mic)
spec_FTM = MultiSTFT(wav[:, :M], n_fft=n_fft)

separater.file_id = input_fname.split("/")[-1].split(".")[0]
separater.load_spectrogram(spec_FTM, sample_rate)
separater.solve(
    n_iter=n_iter,
    save_dir="./",
    save_likelihood=False,
    save_param=False,
    save_wav=True,
    interval_save=5,
)

Update FastMNMF2_IP-M=2-S=5-F=513-K=2-init=circular-g=0.05-bit=64-intv_norm=10  30 times ...


100%|██████████| 30/30 [04:26<00:00,  8.89s/it]


Update FastMNMF2_IP-M=2-S=5-F=513-K=64-init=twostep_30it-g=0.05-bit=64-intv_norm=10-ID=test_audio  70 times ...


100%|██████████| 70/70 [09:00<00:00,  7.72s/it]


In [5]:
input_fname = "FastMNMF2_IP-sep-M=2-S=5-F=513-K=64-init=twostep_30it-g=0.05-bit=64-intv_norm=10-ID=test_audio-100.wav"
wav, sample_rate = torchaudio.load(input_fname, channels_first=False)
wav = wav.t()
for i in range(n_source):
    sf.write(f"sep_{i}.wav", wav[i].numpy(), sample_rate)
